In [169]:
import pandas as pd
import nltk
import re

In [170]:
df = pd.read_csv("../job_scrape_master.csv")
df.head(100)

,scrape_time,job_title,publisher,time_posted,salary,job_type,desc
0,29-Apr-2022 T11:46,Data Engineer,ITCONNECTUS PTE. LTD.,16 hours ago,$5K–$13K a month,Full–time,Immediate joiners\n\nRequirements\n• Technical...
1,29-Apr-2022 T11:46,Data Engineer,ITCONNECTUS PTE. LTD.,16 hours ago,$5K–$13K a month,Full–time,Immediate joiners\n\nRequirements\n• Technical...
2,29-Apr-2022 T11:46,Data Engineer,Asm Technology Singapore Pte Ltd,16 hours ago,NaN,Full–time,ASM Data Engineer\n\nAs ASMPT is rapidly growi...
3,29-Apr-2022 T11:46,"Principal Data Engineer, Data Engineering",GovTech Singapore,2 hours ago,NaN,Full–time,The Government Technology Agency (GovTech) aim...
4,29-Apr-2022 T11:46,Data Engineer,Thakral One,16 hours ago,$8.5K–$10.5K a month,Full–time,• Work with Banking team to understand existin...
...,...,...,...,...,...,...,...
95,29-Apr-2022 T11:47,[LTA-RAOM] ANALYST ENGINEER (DATA & SYS. STRAT...,Land Transport Authority (LTA) Singapore,6 hours ago,NaN,Full–time,You will be part of the Digital asset manageme...
96,29-Apr-2022 T11:47,Software Engineer - (Data Engineering),Goldman Sachs,16 hours ago,NaN,Full–time,OUR IMPACT\n\nSeeking a data focused software ...
97,29-Apr-2022 T11:47,"Senior Associate, Machine Learning Engineer, G...",Dbs Bank Ltd.,8 hours ago,NaN,Full–time,Business Function\n\nGroup Technology and Oper...
98,29-Apr-2022 T11:47,"Facilities Engineer (Data Centre, office hours)",Randstad Singapore,6 hours ago,NaN,Full–time,About The Job\n\nThis is a company whose exper...


In [168]:
#deduplicate
df = df.drop_duplicates(subset=['job_title','publisher'])
df = df.drop_duplicates(subset=['desc'])
df.to_csv("job_scrape_dedup.csv",index=False)

In [164]:
keywords = pd.read_csv("keywords.csv")
terms = list(keywords['variant'])
word_freq_dict = {}
keep_chars = ' -+'
for i,row in df.iterrows():
    desc = re.sub(r'[^\w'+keep_chars+']', ' ',row['desc'])
    for term in terms:
        if term.lower() not in word_freq_dict.keys():
            word_freq_dict[term.lower()] = 0
        if term in desc.lower():
            if term.lower() not in word_freq_dict.keys():
                word_freq_dict[term.lower()] = 1
            else:
                word_freq_dict[term.lower()] += 1
            continue

word_df = pd.DataFrame.from_dict(word_freq_dict,orient='index',columns=['count'])
word_df = word_df.rename_axis('term').reset_index()
word_df = word_df.sort_values('count',ascending=False).reset_index(drop=True)

word_df['term_agg'] = word_df['term'].apply(lambda x:keywords[keywords['variant']==x]['term'].tolist()[0])
word_df['type'] = word_df['term'].apply(lambda x:keywords[keywords['variant']==x]['type'].tolist()[0])
word_df = word_df.groupby(['term_agg','type']).sum().sort_values('count',ascending=False).reset_index()
word_df.to_csv("words.csv",index=False)